In [1]:
import numpy as np
import pandas as pd
from dowhy import CausalModel
import graphviz
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
# EconML imports
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter
import seaborn as sns
import dowhy.plotter
%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')



# The Direct Acyclic Graph

G=graphviz.Digraph()
G.edge("ta_h" , "tr")
G.edge("vel_h" , "ta_h")
G.edge("rh" , "ta_h")
G.edge("ta_h" , "clo")
G.edge("ta_h" , "thermal_sensation")
G.edge("vel_h" , "tr")
G.edge("rh" , "tr")
G.edge("tr" , "clo")
G.edge("tr" , "thermal_sensation")
G.edge("vel_h" , "clo")
G.edge("vel_h" , "thermal_sensation")
G.edge("rh" , "clo")
G.edge("rh" , "thermal_sensation")
G.edge("met" , "thermal_sensation")
G.edge("clo" , "thermal_sensation")



# print(dot.source)

G.format = 'pdf'
G.render(directory='DAG', view = False).replace('\\', '/')



'DAG/Digraph.gv.pdf'

In [2]:
# Reading the data
data = pd.read_excel("data.xlsx")

data_for_causal = data[["ta_h","tr", "vel_h", "rh", "met", "clo", "thermal_sensation"]]
data_for_causal = data_for_causal.dropna()

In [3]:

len(data['country'].unique())

In [4]:
data_for_causal.describe()

,ta_h,tr,vel_h,rh,met,clo,thermal_sensation
count,14861.000000,14861.000000,14861.000000,14861.000000,14861.000000,14861.000000,14861.000000
mean,24.481171,24.347124,0.202645,51.515186,1.209126,0.587638,-0.019958
std,3.026901,3.159064,0.250611,14.457996,0.205215,0.291305,1.267191
min,10.100000,8.500000,0.000000,8.900000,0.700000,0.040000,-3.000000
25%,22.660000,22.485790,0.080000,41.195754,1.100000,0.393840,-1.000000
50%,23.800000,23.600000,0.138955,53.052908,1.200000,0.530000,0.000000
75%,26.600000,26.238518,0.250000,62.200000,1.200000,0.710000,1.000000
max,34.600000,36.300000,6.540000,89.000000,3.100000,2.870000,3.000000


In [5]:
# Initializing causal model
model = CausalModel(data=data_for_causal,
                     graph=G.source.replace("\t", ' ').replace("\n", ' '),
                     treatment="ta_h",
                     outcome="thermal_sensation")

# Identifying the estimation method
identified_estimand= model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                                  
──────(E[thermal_sensation|rh,vel_h])
d[taₕ]                               
Estimand assumption 1, Unconfoundedness: If U→{ta_h} and U→thermal_sensation then P(thermal_sensation|ta_h,rh,vel_h,U) = P(thermal_sensation|ta_h,rh,vel_h)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [6]:
random_state = 120
dml_estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.econml.dml.DML", # Calling EconMl double machine learning algorithm
                                   #   control_value = 0,
                                     treatment_value = 2,
                                    #  target_units = 'ate',
                                     confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              'model_t': GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              "model_final":LassoCV(fit_intercept=False, random_state=random_state),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True),
                                                              'random_state':random_state},
                                               "fit_params":{}})
print(dml_estimate.value)

0.344423708569441


In [ ]:
# Random cause
res_random=model.refute_estimate(identified_estimand, dml_estimate, method_name="random_common_cause", random_seed=123)
print(res_random)

# Add Unobserved Common Causes
res_unobserved=model.refute_estimate(identified_estimand, dml_estimate, method_name="add_unobserved_common_cause",
                                     confounders_effect_on_treatment="linear", confounders_effect_on_outcome="linear",
                                    effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02, random_seed=123)
print(res_unobserved)

# Placebo Treatment
res_placebo=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="placebo_treatment_refuter", random_seed=123)
print(res_placebo)


# Data Subsets Validation
res_subset=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="data_subset_refuter", subset_fraction=0.8,
        num_simulations=10, random_seed=123)
print(res_subset)